In [25]:
import numpy as np
import pandas as pd
import json

In [26]:
with open('data.json', 'r') as f:
    labels = json.load(f)
labels

{'Boy': 0,
 'Can': 1,
 'Eat': 2,
 'Fine': 3,
 'Girl': 4,
 'Help': 5,
 'How': 6,
 'Hungry': 7,
 'I': 8,
 'Mother': 9,
 'Namaste': 10,
 'Name': 11,
 'Parents': 12,
 'Sister': 13,
 'Sleep': 14,
 'This': 15,
 'You': 16}

In [27]:
data=pd.read_csv('dataset.csv')
data.head()

,Class Label,X0001,Y0001,Z0001,X0101,Y0101,Z0101,X0201,Y0201,Z0201,...,Z6219,X6319,Y6319,Z6319,X6419,Y6419,Z6419,X6519,Y6519,Z6519
0,Boy,1.0,0.547994,0.000000,1.0,0.547994,0.000000,1.0,0.547994,0.000000,...,0.071914,0.272580,0.322938,0.127316,0.228388,0.264486,0.084617,0.265310,0.313749,0.177373
1,Boy,1.0,0.857104,0.379246,1.0,0.857104,0.379246,1.0,0.857104,0.379246,...,0.265342,0.458347,0.407922,0.230979,0.368018,0.479731,0.278366,0.415877,0.466096,0.319931
2,Boy,1.0,0.852530,0.358460,1.0,0.852530,0.358460,1.0,0.852530,0.358460,...,0.252582,0.462702,0.409546,0.237752,0.368986,0.483866,0.267000,0.418716,0.471061,0.319184
3,Boy,1.0,0.778636,0.000000,1.0,0.778636,0.000000,1.0,0.778636,0.000000,...,0.003979,0.426772,0.485623,0.153090,0.321029,0.400402,0.032188,0.410688,0.467018,0.270999
4,Boy,1.0,0.589883,0.119620,1.0,0.589883,0.119620,1.0,0.589883,0.119620,...,0.380987,0.487479,0.526593,0.386855,0.465436,0.526274,0.390475,0.482003,0.520124,0.418845


In [28]:
Y = data['Class Label']
Y

0      Boy
1      Boy
2      Boy
3      Boy
4      Boy
      ... 
717    You
718    You
719    You
720    You
721    You
Name: Class Label, Length: 722, dtype: object

In [29]:
Y = Y.map(labels)
type(Y)

pandas.core.series.Series

In [30]:
# Y=pd.DataFrame(Y)
Y=np.array(Y)
Y

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,
        5,  5,  5,  5,  5

In [31]:
# from tensorflow.keras.utils import to_categorical
# Y=to_categorical(Y).astype(int)
# Y

In [32]:
Y.shape

(722,)

In [33]:
data.drop(['Class Label'], axis=1, inplace=True)
data.head()

,X0001,Y0001,Z0001,X0101,Y0101,Z0101,X0201,Y0201,Z0201,X0301,...,Z6219,X6319,Y6319,Z6319,X6419,Y6419,Z6419,X6519,Y6519,Z6519
0,1.0,0.547994,0.000000,1.0,0.547994,0.000000,1.0,0.547994,0.000000,1.0,...,0.071914,0.272580,0.322938,0.127316,0.228388,0.264486,0.084617,0.265310,0.313749,0.177373
1,1.0,0.857104,0.379246,1.0,0.857104,0.379246,1.0,0.857104,0.379246,1.0,...,0.265342,0.458347,0.407922,0.230979,0.368018,0.479731,0.278366,0.415877,0.466096,0.319931
2,1.0,0.852530,0.358460,1.0,0.852530,0.358460,1.0,0.852530,0.358460,1.0,...,0.252582,0.462702,0.409546,0.237752,0.368986,0.483866,0.267000,0.418716,0.471061,0.319184
3,1.0,0.778636,0.000000,1.0,0.778636,0.000000,1.0,0.778636,0.000000,1.0,...,0.003979,0.426772,0.485623,0.153090,0.321029,0.400402,0.032188,0.410688,0.467018,0.270999
4,1.0,0.589883,0.119620,1.0,0.589883,0.119620,1.0,0.589883,0.119620,1.0,...,0.380987,0.487479,0.526593,0.386855,0.465436,0.526274,0.390475,0.482003,0.520124,0.418845


In [34]:
data.shape

(722, 3762)

In [35]:
# X=[]
for i in range(data.shape[0]):
    # for j in range(data.shape[1]):
    #     if data.iloc[i][j].isna():
    #         data.iloc[i][j] = np.mean(data.iloc[i])
    data.iloc[i].fillna(data.iloc[i].mean(), inplace=True)
    # X.append(np.array(data.iloc[0][1:]))

C:\Users\Dell\AppData\Local\Temp\ipykernel_6556\1369908113.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.iloc[i].fillna(data.iloc[i].mean(), inplace=True)


In [36]:
data.isna().sum()

X0001    0
Y0001    0
Z0001    0
X0101    0
Y0101    0
        ..
Y6419    0
Z6419    0
X6519    0
Y6519    0
Z6519    0
Length: 3762, dtype: int64

In [37]:
# X=np.array(X)
# X.shape
type(data.iloc[0])

pandas.core.series.Series

SVC

In [38]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, Y, test_size=0.2, random_state=42)

In [39]:
# from sklearn.svm import SVC
# model=SVC(kernel='rbf', C=100, gamma=0.001, random_state=42, verbose=True, max_iter=1000, decision_function_shape='ovo')
# model.fit(x_train, y_train)
# pred=model.predict(x_test)

In [40]:
from sklearn.metrics import classification_report, confusion_matrix
# # print(confusion_matrix(y_test, pred))
# print(classification_report(y_test, pred))

In [41]:
# from sklearn.model_selection import GridSearchCV
# param_grid={'C': [0.1,1,10,100,1000], 'gamma': [1,0.1,0.01,0.001,0.0001]}
# grid=GridSearchCV(SVC(), param_grid, verbose=5)

In [42]:
# grid.fit(x_train, y_train)

In [43]:
# grid.best_params_

In [44]:
# grid_pred=grid.predict(x_test)

Random Forest

In [45]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(x_train, y_train)

# Make predictions on the test set
pred = model.predict(x_test)

In [46]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      0.78      0.88         9
           1       0.86      0.71      0.77        17
           2       1.00      1.00      1.00         8
           3       0.78      1.00      0.88         7
           4       1.00      0.62      0.77         8
           5       0.80      0.44      0.57         9
           6       0.33      0.44      0.38         9
           7       0.78      0.88      0.82         8
           8       1.00      0.56      0.71         9
           9       1.00      0.75      0.86         8
          10       0.83      0.71      0.77         7
          11       0.77      1.00      0.87        10
          12       0.67      1.00      0.80         2
          13       1.00      1.00      1.00         2
          14       0.69      0.90      0.78        10
          15       0.46      0.86      0.60         7
          16       0.73      0.73      0.73        15

    accuracy              

In [47]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

XGBoost

In [48]:
import xgboost as xgb

dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)

# Train the model
params = {
    'objective': 'multi:softmax',  # Specify multi-class classification
    'num_class': 3,  # Number of classes in the dataset
    'max_depth': 3,
    'eta': 0.1,
    'gamma': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eval_metric': 'merror'  # Multiclass classification error rate
}
bst = xgb.train(params, dtrain, 100)

# Make predictions
preds = bst.predict(dtest)


XGBoostError: [11:48:14] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\multiclass_obj.cu:123: SoftmaxMultiClassObj: label must be in [0, num_class).